<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/Writing-a-thesis/blob/main/crossvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### crossvalidation.py

## 교차검증을 활용한 모델학습 및 성능평가

def p(str):
    print(str, '\n')


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('/content/drive/MyDrive/KOREAIT/train.csv')
p(df.head())

# 패쳐 / 레이블
features = df.drop('Survived', axis=1) # 패쳐
labels = df['Survived'] # 레이블
p(features)
p(labels)

## features 데이터 전처리
# features['Age'].fillna(df['Age'].mean(), inplace=True)
#
# features['Cabin'].fillna('N', inplace= True)
# features['Embarked'].fillna('N', inplace= True)
# p(df.isna().sum())
# features.drop('Name', axis=1, inplace=True)
#
# def encode_features(df, features):
#     for i in features:
#         le = LabelEncoder()
#         le = le.fit(df[i])
#         df[i] = le.transform(df[i])
#     return df # feature 들이 레이블 인코딩된 데이터 프레임 반환
#
# p(encode_features(features, ['Sex','Ticket', 'Cabin', 'Embarked']))
#
#
#
#
#
#
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import cross_validate
# from sklearn.ensemble import RandomForestClassifier
#
#
#
# model = RandomForestClassifier(n_estimators=100, random_state=42)
# cv_results = cross_validate(model, features, labels, cv=5, scoring=['accuracy', 'precision', 'recall'], return_train_score=True)
#
# # 교차 검증 결과 출력
# print(f"Test Accuracy Scores: {cv_results['test_accuracy']}")
# print(f"Mean Test Accuracy: {cv_results['test_accuracy'].mean()}")
#
# print(f"Test Precision Scores: {cv_results['test_precision']}")
# print(f"Mean Test Precision: {cv_results['test_precision'].mean()}")
#
# print(f"Test Recall Scores: {cv_results['test_recall']}")
# print(f"Mean Test Recall: {cv_results['test_recall'].mean()}")
#
# print(f"Train Accuracy Scores: {cv_results['train_accuracy']}")
# print(f"Mean Train Accuracy: {cv_results['train_accuracy'].mean()}")
#
# print(f"Fit Times: {cv_results['fit_time']}")
# print(f"Score Times: {cv_results['score_time']}")
#
#
# # 교차 검증 수행
# # 5-폴드 교차 검증을 사용하여 모델의 성능 평가
# scores = cross_val_score(model,  features, labels, cv=5)
#
# # 교차 검증 결과 출력
# print(f"Cross-Validation Scores: {scores}")
# print(f"Mean Accuracy: {scores.mean()}")


#############
p(features.isnull().sum())
# 결측치 제거
features['Age'].fillna(features['Age'].mean(), inplace=True)
features['Cabin'].fillna('N', inplace= True)
features['Embarked'].fillna('N', inplace= True)

# 불필요한 속성 제거(PassengerId, Name, Ticket)
features = features.drop(['PassengerId', 'Name', 'Ticket'],axis=1)

p(features)

# 레이블 인코딩 진행
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
features['Sex'] = le.fit_transform(features['Sex'])
features['Embarked'] = le.fit_transform(features['Embarked'])
features['Cabin'] = le.fit_transform(features['Cabin'])
p(features)

# 훈련세트와 테스트세트 분할
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import StratifiedKFold



X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

rf = RandomForestClassifier(random_state=42)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=skf, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train, y_train)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_}")

best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print(f"Test Set Accuracy: {test_accuracy}")










   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S   



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


dt = DecisionTreeClassifier(random_state=42)
# p(dt)

# 모델 학습
dt.fit(X_train, y_train)

# 예측치
pred = dt.predict(X_test)
p(pred)

# 정확도
p(f'정확도 : {accuracy_score(y_test, pred)}')

# 교차검증

from sklearn.model_selection import cross_val_score
# dt : 데이터 프레임 , features : 패처 , labels : 레이블, cv: 폴드수
scores = cross_val_score(dt, features, labels, cv= 10)

# 교차검증마다의 정확도 구하기
# iter_count : 반복횟수, accuracy : 정확도
# enumerate(scores):교차검증결과를 열거
for iter_count, accuracy in enumerate(scores):
    p(f'{iter_count+1}번째 교차검증 정확도:{accuracy}')

[0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0
 1 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1
 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1
 0 1 1 0 0 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0
 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1] 

정확도 : 0.7988826815642458 

1번째 교차검증 정확도:0.7444444444444445 

2번째 교차검증 정확도:0.7640449438202247 

3번째 교차검증 정확도:0.7191011235955056 

4번째 교차검증 정확도:0.7865168539325843 

5번째 교차검증 정확도:0.8314606741573034 

6번째 교차검증 정확도:0.7865168539325843 

7번째 교차검증 정확도:0.797752808988764 

8번째 교차검증 정확도:0.7191011235955056 

9번째 교차검증 정확도:0.8651685393258427 

10번째 교차검증 정확도:0.7865168539325843 

